In [5]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize, LabelBinarizer
import sys
sys.path.append('../')
from modules import utils, constants

In [6]:
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

#### The datasets

In [7]:
train_df = pd.read_csv('../data/18_jan/train_set_basic.csv')
train_df = train_df.fillna(-1)
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,acute_pericarditis,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody
0,1,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,1,0
3,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,1,0,1,0,0,0,0,0,1,0,...,0,1,1,0,1,0,1,0,0,0


In [8]:
test_df = pd.read_csv('../data/18_jan/test_set_basic.csv')
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]
X_test.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,acute_pericarditis,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody
0,1,0,0,0,0,1,0,1,0,0,...,0,1,0,0,1,0,1,0,0,0
1,1,0,0,0,0,1,1,0,1,0,...,0,1,0,0,0,0,0,0,1,0
2,1,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
3,1,1,0,0,0,1,0,0,1,0,...,0,0,1,1,1,0,0,0,0,0
4,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 23), (14000, 23), (56000,), (14000,))

In [10]:
from sklearn.svm import SVC
svm_model = SVC(kernel='linear', C=1, decision_function_shape='ovo', random_state=constants.SEED).fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
test_df_svm = pd.DataFrame()
test_df_svm['y_actual'] = y_test
test_df_svm['y_pred'] = y_pred_svm
test_df_svm.isna().sum()

y_actual    0
y_pred      0
dtype: int64

In [13]:
acc_svm, prec_svm, recall_svm, f1_svm = utils.test_binary(test_df_svm['y_actual'], test_df_svm['y_pred'])
acc_svm, prec_svm, recall_svm, f1_svm

(96.87857142857142, 97.9037703995498, 96.02594176900786, 96.95576454197145)

In [12]:
acc_svm2, f1_svm2, roc_auc_svm2 = utils.test(test_df_svm['y_actual'], test_df_svm['y_pred'])
acc_svm2, f1_svm2, roc_auc_svm2

(96.87857142857142, 96.87656310645659, 96.90975749786095)

#### Performance after normalization